In [5]:
#Observations
#With the pandemic, 
#1. Long-term unemployment (people who are out of work 27 weeks or longer as a percent of total unemployed) 
#   has increased 8-fold.
#2. The Hispanic/Latino community had the highest unemployment at the onset of the pandemic, but has seen the
#   most dramatic improvement since.
#3. The African-American group continues to be the most impacted by the pandemic.
#4. It should come as no surprise that unemployment, during periods where there is an extraneous shock to the system 
#   such as the pandemic, is reflected as an outlier.  
#   The question is whether the outlier sets the new boundaries over time.

In [6]:
#load modules
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import json 
import requests
from config import api_key


ModuleNotFoundError: No module named 'config'

In [ ]:
#BLS url
endpoint = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

#timeseries of monthly unemployment data to query
series = ['LNS14000000','LNS14000006','LNS14000009','LNS14032183','LNS14000003','LNS13025703'] 
startyear = '2018'
endyear = '2020'


#setup json request
headers = {'Content-type': 'application/json'}
data = json.dumps(
    {"seriesid": series,
     "registrationkey": api_key,
     "startyear":startyear, 
     "endyear":endyear})

stat = requests.post(endpoint, data=data, headers=headers)
stat_data = json.loads(stat.text)
#stat = requests.post(endpoint, data=data, headers=headers)


#setup lists to hold stats
seriesid = []
year= []
period = []
periodName = []
value = []

for series in stat_data['Results']['series']:
    seriesId = series['seriesID']
    for data in series['data']:
        year.append(data['year'])
        period.append(data['period'])
        periodName.append(data['periodName'])
        value.append(data['value'])
        seriesid.append(series['seriesID'])
 

In [ ]:
#create dataframe
unemployment_df = pd.DataFrame({
    "Series":seriesid,
    "Year": year,
    "Month Code": period,
    "Month": periodName,
    "Unemployment Rate": value
})

unemployment_df.head()

In [4]:
#Add combined Mth and Year column
unemployment_df['Mth/Year'] = pd.to_datetime(unemployment_df['Month'].astype(str)+unemployment_df['Year'], format='%B%Y')

#convert Unemployment Rate to numeric to get max and min
unemployment_df['Unemployment Rate'] = pd.to_numeric(unemployment_df['Unemployment Rate'])

#sort values
unemployment_df = unemployment_df.sort_values(by=['Series','Mth/Year'], ascending=[True,True], inplace=False, ignore_index=True)

#verify sort and check min, max
unemployment_df.head(), unemployment_df['Unemployment Rate'].min(), unemployment_df['Unemployment Rate'].max()

NameError: name 'unemployment_df' is not defined

In [3]:
#drop Month and Year
unemployment_df = unemployment_df[['Series', 'Month Code', 'Unemployment Rate', 'Mth/Year']]

unemployment.head()

NameError: name 'unemployment_df' is not defined

In [ ]:
#series description
#'LNS14000000' is 'Seasonal Total Unemployment Rate'
#'LNS14000006' is 'Unemployment-AfricanAmerican'
#'LNS14000009' is 'Unemployment-Hispanic'
#'LNS14032183' is 'Unemployment-Asian'
#'LNS14000003' is 'Unemployment-White'
#'LNS13025703' is 'Long-Term Unemployment Rate'

In [ ]:
#Isolating the unemployment rate
unerate = unemployment.loc[unemployment['Series'] == 'LNS14000000',:]
uneafam = unemployment.loc[unemployment['Series'] == 'LNS14000006',:]
unehisp = unemployment.loc[unemployment['Series'] == 'LNS14000009',:]
uneasian = unemployment.loc[unemployment['Series'] =='LNS14032183',:]
unewhite = unemployment.loc[unemployment['Series'] =='LNS14000003',:]
ltunemp = unemployment.loc[unemployment['Series'] == 'LNS13025703',:]


In [ ]:
#line graph of x and y axis
plt.figure(figsize=(10,4))

#setting x axis 
years = ltunemp['Mth/Year']

#setting y axis 
total, = plt.plot(years, ltunemp['Unemployment Rate'], color='red', label = 'Long-Term Unemployment',linewidth =1, linestyle = '--') 
unemp, = plt.plot(years, unerate['Unemployment Rate'], color='black', label = 'Seasonal Total Unemployment',linewidth =1) 
afam, = plt.plot(years, uneafam['Unemployment Rate'], color= 'orange', label = "Seasonal Unemployment=African American", linewidth=1) 
hisp, = plt.plot(years, unehisp['Unemployment Rate'], color= 'green', label = "Seasonal Unemployment=Hispanic/Latino", linewidth=1) 
asian, = plt.plot(years, uneasian['Unemployment Rate'], color= 'indigo', label = "Seasonal Unemployment=Asian", linewidth=1) 
white, = plt.plot(years, unewhite['Unemployment Rate'], color= 'blue', label = "Seasonal Unemployment=Caucasian", linewidth=1)

#setting properties 
plt.yticks(np.arange(round(unemployment['Unemployment Rate'].min()), 
                     round(unemployment['Unemployment Rate'].max()) +1, 2.0))  
plt.xticks(rotation=45)
plt.title("US Unemployment Rate - Long Term and by Ethnic Group")  

#set legend outside of box 
plt.legend(bbox_to_anchor=(1.05, 1),handles=[total, unemp, afam, hisp, asian, white])
 
plt.tight_layout() 
plt.show()

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
ltrate = pd.Series(ltunemp['Unemployment Rate'])

#IQR
ltquartiles = ltrate.quantile([0.25,0.5,0.75])
ltlowerq = ltquartiles[0.25]
ltupperq = ltquartiles[0.75]
ltiqr = ltupperq - ltlowerq
ltlower_bound = ltlowerq - (1.5 * ltiqr)
ltupper_bound = ltupperq + (1.5 * ltiqr)

#outliers
ltoutliers = [rate for rate in ltrate if rate < ltlower_bound or rate > ltupper_bound]

len(ltoutliers)
ltoutliers_distinct = list(set(ltoutliers))
len(ltoutliers_distinct)
ltoutliers_distinct.sort()
ltoutliers_distinct  

ltoutliers_distinct

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
unrate = pd.Series(unerate['Unemployment Rate'])

#IQR
unquartiles = unrate.quantile([0.25,0.5,0.75])
unlowerq = unquartiles[0.25]
unupperq = unquartiles[0.75]
uniqr = unupperq - unlowerq
unlower_bound = unlowerq - (1.5 * uniqr)
unupper_bound = unupperq + (1.5 * uniqr)

#outliers
unoutliers = [rate for rate in unrate if rate < unlower_bound or rate > unupper_bound]

len(unoutliers)
unoutliers_distinct = list(set(unoutliers))
len(unoutliers_distinct)
unoutliers_distinct.sort()
unoutliers_distinct  

unoutliers_distinct

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
afrate = pd.Series(uneafam['Unemployment Rate'])

#IQR
afquartiles = afrate.quantile([0.25,0.5,0.75])
aflowerq = afquartiles[0.25]
afupperq = afquartiles[0.75]
afiqr = afupperq - aflowerq
aflower_bound = aflowerq - (1.5 * afiqr)
afupper_bound = afupperq + (1.5 * afiqr)

#outliers
afoutliers = [rate for rate in afrate if rate < aflower_bound or rate > afupper_bound]

len(afoutliers)
afoutliers_distinct = list(set(afoutliers))
len(afoutliers_distinct)
afoutliers_distinct.sort()
afoutliers_distinct  

afoutliers_distinct

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
hprate = pd.Series(unehisp['Unemployment Rate'])

#IQR
hpquartiles = hprate.quantile([0.25,0.5,0.75])
hplowerq = hpquartiles[0.25]
hpupperq = hpquartiles[0.75]
hpiqr = hpupperq - hplowerq
hplower_bound = hplowerq - (1.5 * hpiqr)
hpupper_bound = hpupperq + (1.5 * hpiqr)

#outliers
hpoutliers = [rate for rate in hprate if rate < hplower_bound or rate > hpupper_bound]

len(hpoutliers)
hpoutliers_distinct = list(set(hpoutliers))
len(hpoutliers_distinct)
hpoutliers_distinct.sort()
hpoutliers_distinct  

hpoutliers_distinct

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
asrate = pd.Series(uneasian['Unemployment Rate'])

#IQR
asquartiles = asrate.quantile([0.25,0.5,0.75])
aslowerq = asquartiles[0.25]
asupperq = asquartiles[0.75]
asiqr = asupperq - aslowerq
aslower_bound = aslowerq - (1.5 * asiqr)
asupper_bound = asupperq + (1.5 * asiqr)

#outliers
asoutliers = [rate for rate in asrate if rate < aslower_bound or rate > asupper_bound]

len(asoutliers)
asoutliers_distinct = list(set(asoutliers))
len(asoutliers_distinct)
asoutliers_distinct.sort()
asoutliers_distinct  

asoutliers_distinct

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine outliers using upper and lower bounds

#LT unemployment
wtrate = pd.Series(unewhite['Unemployment Rate'])

#IQR
wtquartiles = wtrate.quantile([0.25,0.5,0.75])
wtlowerq = wtquartiles[0.25]
wtupperq = wtquartiles[0.75]
wtiqr = wtupperq - wtlowerq
wtlower_bound = wtlowerq - (1.5 * wtiqr)
wtupper_bound = wtupperq + (1.5 * wtiqr)

#outliers
wtoutliers = [rate for rate in wtrate if rate < wtlower_bound or rate > wtupper_bound]

len(wtoutliers)
wtoutliers_distinct = list(set(wtoutliers))
len(wtoutliers_distinct)
wtoutliers_distinct.sort()
wtoutliers_distinct  

wtoutliers_distinct

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
ax1.set_title('Unemployment Rate - Long Term vs Ethnic Groups')
ax1.set_ylabel('Unemployment Rate')
ax1.boxplot([ltrate,unrate, afrate, hprate, asrate, wtrate], showfliers=True)
plt.xticks([1,2,3,4,5,6],['Long Term', 'Seas Total',"Afr/Amer", 'Hispanic','Asian','Caucasian'])
plt.show()

In [ ]:
#retrieve US Covid cases file
#load file
#file source: https://ourworldindata.org/coronavirus/country/united-states?country=~USA
covid_path = "D:\\GroupProject1\\Statistics\\daily-covid-cases-7-day.csv"

#Read the data and remove commas from numbers
covid = pd.read_csv(covid_path,  encoding = 'iso-8859-1', low_memory=False)

covid = covid.loc[(covid['Entity'] == 'United States')]
covid.tail()


In [ ]:
#=pd.to_datetime(covid['Date'].astype(str), format='%B%Y')
covid['Date'] = pd.to_datetime(covid['Date'].astype(str),format='%m/%d/%Y')
covid['Year'] = covid['Date'].dt.year
covid['Month'] = covid['Date'].dt.month
covid.head()

In [ ]:
#covid_df = covid.groupby(['Month'])
mthly_case = []
for case in covid['Month']:
    if case == 1:
        mthly_case.append(covid.iloc[case:3])
mthly_case